# Interval Tree & Sweep Line Patterns

## 1. Interval Tree

An **Interval Tree** is a balanced binary search tree designed to store intervals and efficiently answer **overlap queries**:
- **Query**: Given interval [a, b], find all intervals that overlap with it.
- **Insert/Delete**: Add or remove intervals.
- **Time complexity**: O(log n + k) where k is the number of overlapping intervals.

**Key insight**: Each node stores the **maximum endpoint** of all intervals in its subtree, enabling efficient pruning during traversal.

---

## 2. Sweep Line Algorithm

The **Sweep Line** technique processes geometric problems by imagining a vertical line sweeping across a plane from left to right. Events (start/end of intervals, points) are processed in sorted order.

**Common applications**:
- **Line Segment Intersection**: Detect intersections among multiple line segments.
- **Interval Scheduling**: Find maximum non-overlapping intervals.
- **Rectangle Union**: Compute area covered by overlapping rectangles.
- **Closest Pair**: Find closest pair of points.

**Pattern**:
1. Convert problem into **events** (start/end points).
2. **Sort events** by x-coordinate (or time).
3. **Process events** left-to-right, maintaining active set.
4. Update answer as events are processed.

---

## 3. Common Problems

### Interval Overlap Problems:
- **Meeting Rooms II**: Minimum rooms needed for overlapping meetings.
- **Merge Intervals**: Combine overlapping intervals.
- **Insert Interval**: Insert new interval into sorted list.

### Sweep Line Problems:
- **Rectangle Area**: Total area covered by rectangles.
- **Skyline Problem**: Building skyline silhouette.
- **Event Scheduler**: Maximum concurrent events.

**💡 Key Pattern**: Convert intervals to **events**, sort by time, then sweep!

In [7]:
use std::cmp::Ordering;

// Interval represents a time interval [start, end)
#[derive(Debug, Clone, Copy, PartialEq, Eq)]
pub struct Interval {
    pub start: i32,
    pub end: i32,
}

impl Interval {
    pub fn new(start: i32, end: i32) -> Self {
        Interval { start, end }
    }
}

// IntervalNode represents a node in the interval tree
#[derive(Debug)]
pub struct IntervalNode {
    interval: Interval,
    max_end: i32, // Maximum endpoint in this subtree
    left: Option<Box<IntervalNode>>,
    right: Option<Box<IntervalNode>>,
}

impl IntervalNode {
    fn new(interval: Interval) -> Self {
        IntervalNode {
            interval,
            max_end: interval.end,
            left: None,
            right: None,
        }
    }
}

// IntervalTree represents an interval tree for overlap queries
#[derive(Debug)]
pub struct IntervalTree {
    root: Option<Box<IntervalNode>>,
}

impl IntervalTree {
    pub fn new() -> Self {
        IntervalTree { root: None }
    }

    // Insert an interval into the tree
    pub fn insert(&mut self, interval: Interval) {
        self.root = Self::insert_node(self.root.take(), interval);
    }

    fn insert_node(node: Option<Box<IntervalNode>>, interval: Interval) -> Option<Box<IntervalNode>> {
        match node {
            None => Some(Box::new(IntervalNode::new(interval))),
            Some(mut node) => {
                // Insert based on start time
                if interval.start < node.interval.start {
                    node.left = Self::insert_node(node.left.take(), interval);
                } else {
                    node.right = Self::insert_node(node.right.take(), interval);
                }

                // Update max endpoint
                if interval.end > node.max_end {
                    node.max_end = interval.end;
                }

                Some(node)
            }
        }
    }

    // Search for all intervals that overlap with the given interval
    pub fn search_overlap(&self, interval: Interval) -> Vec<Interval> {
        let mut result = Vec::new();
        Self::search_overlap_helper(&self.root, interval, &mut result);
        result
    }

    fn search_overlap_helper(
        node: &Option<Box<IntervalNode>>,
        interval: Interval,
        result: &mut Vec<Interval>,
    ) {
        if let Some(node) = node {
            // Check if current interval overlaps
            if Self::do_overlap(node.interval, interval) {
                result.push(node.interval);
            }

            // If left subtree's max endpoint >= interval.start, there might be overlap in left
            if let Some(ref left) = node.left {
                if left.max_end >= interval.start {
                    Self::search_overlap_helper(&node.left, interval, result);
                }
            }

            // Always check right subtree if current interval's start <= interval.end
            if node.interval.start <= interval.end {
                Self::search_overlap_helper(&node.right, interval, result);
            }
        }
    }

    // Check if two intervals overlap
    fn do_overlap(a: Interval, b: Interval) -> bool {
        a.start < b.end && b.start < a.end
    }
}

// Event represents a sweep line event
#[derive(Debug, Clone, Copy)]
pub struct Event {
    time: i32,
    event_type: EventType, // Start or End
    interval: Interval,
}

#[derive(Debug, Clone, Copy, PartialEq, Eq)]
pub enum EventType {
    Start,
    End,
}

impl Event {
    fn new(time: i32, event_type: EventType, interval: Interval) -> Self {
        Event {
            time,
            event_type,
            interval,
        }
    }
}

// SweepLine solves interval problems using sweep line algorithm
#[derive(Debug)]
pub struct SweepLine {
    events: Vec<Event>,
}

impl SweepLine {
    pub fn new() -> Self {
        SweepLine { events: Vec::new() }
    }

    // Add an interval to the sweep line
    pub fn add_interval(&mut self, interval: Interval) {
        self.events.push(Event::new(interval.start, EventType::Start, interval));
        self.events.push(Event::new(interval.end, EventType::End, interval));
    }

    // Find the maximum number of overlapping intervals
    pub fn max_concurrent_intervals(&mut self) -> usize {
        if self.events.is_empty() {
            return 0;
        }

        // Sort events by time, with end events before start events at same time
        self.events.sort_by(|a, b| {
            match a.time.cmp(&b.time) {
                Ordering::Equal => {
                    // End events before start events at same time
                    match (a.event_type, b.event_type) {
                        (EventType::End, EventType::Start) => Ordering::Less,
                        (EventType::Start, EventType::End) => Ordering::Greater,
                        _ => Ordering::Equal,
                    }
                }
                other => other,
            }
        });

        let mut max_count = 0;
        let mut current_count = 0;

        for event in &self.events {
            match event.event_type {
                EventType::Start => {
                    current_count += 1;
                    max_count = max_count.max(current_count);
                }
                EventType::End => {
                    current_count -= 1;
                }
            }
        }

        max_count
    }

    // Merge overlapping intervals
    pub fn merge_intervals(&self) -> Vec<Interval> {
        if self.events.is_empty() {
            return Vec::new();
        }

        // Extract intervals and sort by start time
        let mut intervals: Vec<Interval> = self.events
            .iter()
            .filter(|e| e.event_type == EventType::Start)
            .map(|e| e.interval)
            .collect();

        intervals.sort_by_key(|interval| interval.start);

        let mut merged: Vec<Interval> = Vec::new();
        for interval in intervals {
            if merged.is_empty() || merged.last().unwrap().end < interval.start {
                // No overlap, add new interval
                merged.push(interval);
            } else {
                // Overlap, merge with last interval
                let last = merged.last_mut().unwrap();
                last.end = last.end.max(interval.end);
            }
        }

        merged
    }
}

// Point represents a 2D point for sweep line problems
#[derive(Debug, Clone, Copy)]
pub struct Point {
    pub x: i32,
    pub y: i32,
}

// Rectangle represents a rectangle for area calculation
#[derive(Debug, Clone, Copy)]
pub struct Rectangle {
    pub x1: i32,
    pub y1: i32,
    pub x2: i32,
    pub y2: i32,
}

impl Rectangle {
    pub fn new(x1: i32, y1: i32, x2: i32, y2: i32) -> Self {
        Rectangle { x1, y1, x2, y2 }
    }
}

// SweepLineGeometry handles geometric sweep line problems
#[derive(Debug)]
pub struct SweepLineGeometry {
    rectangles: Vec<Rectangle>,
}

impl SweepLineGeometry {
    pub fn new() -> Self {
        SweepLineGeometry {
            rectangles: Vec::new(),
        }
    }

    // Add a rectangle to the collection
    pub fn add_rectangle(&mut self, rect: Rectangle) {
        self.rectangles.push(rect);
    }

    // Calculate total area covered by rectangles (simplified version)
    pub fn rectangle_area_union(&self) -> i32 {
        if self.rectangles.is_empty() {
            return 0;
        }

        // Create events for vertical edges
        #[derive(Debug, Clone, Copy)]
        struct VerticalEvent {
            x: i32,
            y1: i32,
            y2: i32,
            event_type: EventType,
        }

        let mut events = Vec::new();
        for rect in &self.rectangles {
            events.push(VerticalEvent {
                x: rect.x1,
                y1: rect.y1,
                y2: rect.y2,
                event_type: EventType::Start,
            });
            events.push(VerticalEvent {
                x: rect.x2,
                y1: rect.y1,
                y2: rect.y2,
                event_type: EventType::End,
            });
        }

        // Sort events by X coordinate
        events.sort_by(|a, b| {
            match a.x.cmp(&b.x) {
                Ordering::Equal => {
                    // Start events before end events at same x
                    match (a.event_type, b.event_type) {
                        (EventType::Start, EventType::End) => Ordering::Less,
                        (EventType::End, EventType::Start) => Ordering::Greater,
                        _ => Ordering::Equal,
                    }
                }
                other => other,
            }
        });

        let mut total_area = 0;
        let mut prev_x = events[0].x;
        let mut active_intervals = Vec::new();

        for event in events {
            // Calculate area contribution from previous sweep
            if event.x > prev_x {
                total_area += (event.x - prev_x) * self.calculate_active_height(&active_intervals);
            }

            // Update active intervals
            match event.event_type {
                EventType::Start => {
                    active_intervals.push(Interval::new(event.y1, event.y2));
                }
                EventType::End => {
                    // Remove the interval
                    if let Some(pos) = active_intervals
                        .iter()
                        .position(|&interval| interval.start == event.y1 && interval.end == event.y2)
                    {
                        active_intervals.remove(pos);
                    }
                }
            }

            prev_x = event.x;
        }

        total_area
    }

    // Calculate the total height covered by active intervals
    fn calculate_active_height(&self, intervals: &[Interval]) -> i32 {
        if intervals.is_empty() {
            return 0;
        }

        // Sort intervals by start
        let mut sorted_intervals = intervals.to_vec();
        sorted_intervals.sort_by_key(|interval| interval.start);

        // Merge overlapping intervals and sum heights
        let mut total_height = 0;
        let mut current_end = sorted_intervals[0].start;

        for interval in sorted_intervals {
            if interval.start > current_end {
                // No overlap
                total_height += interval.end - interval.start;
                current_end = interval.end;
            } else if interval.end > current_end {
                // Partial overlap
                total_height += interval.end - current_end;
                current_end = interval.end;
            }
            // Complete overlap: no additional height
        }

        total_height
    }
}

## Examples

Below are examples showing interval tree and sweep line algorithm usage.

In [8]:
// Interval Tree Example
let mut tree = IntervalTree::new();

// Insert intervals
let intervals = vec![
    Interval::new(1, 4),
    Interval::new(2, 6),
    Interval::new(8, 10),
    Interval::new(3, 5),
    Interval::new(15, 18),
];

for interval in intervals {
    tree.insert(interval);
}

// Search for overlapping intervals
let query = Interval::new(4, 7);
let overlaps = tree.search_overlap(query);
print!("Intervals overlapping with [{}, {}): ", query.start, query.end);
for overlap in overlaps {
    print!("[{}, {}) ", overlap.start, overlap.end);
}
println!();

Intervals overlapping with [4, 7): [2, 6) [3, 5) 


In [9]:
// Sweep Line Example - Maximum Concurrent Intervals
let mut sweep = SweepLine::new();

let meetings = vec![
    Interval::new(1, 3),
    Interval::new(2, 4),
    Interval::new(3, 6),
    Interval::new(5, 7),
    Interval::new(8, 9),
];

for meeting in meetings {
    sweep.add_interval(meeting);
}

let max_rooms = sweep.max_concurrent_intervals();
println!("Maximum meeting rooms needed: {}", max_rooms);

// Merge overlapping intervals
let merged = sweep.merge_intervals();
print!("Merged intervals: ");
for interval in merged {
    print!("[{}, {}) ", interval.start, interval.end);
}
println!();

Maximum meeting rooms needed: 2
Merged intervals: [1, 7) [8, 9) 
Merged intervals: [1, 7) [8, 9) 


In [10]:
// Sweep Line Geometry Example - Rectangle Area Union
let mut geom = SweepLineGeometry::new();

let rectangles = vec![
    Rectangle::new(1, 1, 3, 3), // Rectangle from (1,1) to (3,3)
    Rectangle::new(2, 2, 4, 4), // Rectangle from (2,2) to (4,4) - overlaps with first
    Rectangle::new(5, 1, 6, 2), // Rectangle from (5,1) to (6,2) - separate
];

for rect in rectangles {
    geom.add_rectangle(rect);
}

let total_area = geom.rectangle_area_union();
println!("Total area covered by rectangles: {}", total_area);

Total area covered by rectangles: 8
